In [138]:
%pylab inline

import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
import sklearn as sk
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


/home/prabhakar/.local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imread', 'test', 'seed']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Setup the data

In [143]:
seed = 128
rng = np.random.RandomState(seed)

In [144]:
root_dir = os.path.abspath('..')
data_dir = os.path.join(root_dir, 'data')

# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)

True

In [145]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype=np.float32)
test = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, dtype=np.float32)

In [146]:
train_x = train.iloc[:, :-1].values
test_x = test.iloc[:, :].values

In [147]:
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train.popularity.values[:split_size], train.popularity.values[split_size:]

# Auxilary functions

In [148]:
def dense_to_one_hot(labels_dense, num_classes=4):
    """Convert class labels from scalars to one-hot vectors"""
    s = pd.Series(list(range(1,num_classes+1,1)))
    labels_one_hot = pd.get_dummies(labels_dense)
    
#     num_labels = labels_dense.shape[0]
#     index_offset = np.arange(num_labels) * num_classes
#     labels_one_hot = np.zeros((num_labels, num_classes))
#     labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1, with mean centered"""
    temp_batch = (unclean_batch_x-unclean_batch_x.min()) / (unclean_batch_x.max()-unclean_batch_x.min())
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]].reshape(-1, input_num_units)
    batch_x = preproc(batch_x)
    
    if dataset_name == 'train':
        batch_y = eval(dataset_name).ix[batch_mask, 'popularity'].values
        batch_y = dense_to_one_hot(batch_y)
        
    return batch_x, batch_y

# Neural Net definition

In [182]:
### set all variables

# number of neurons in each layer
input_num_units = 6
hidden_num_units_1 = 250
hidden_num_units_2 = 250
hidden_num_units_3 = 250
hidden_num_units_4 = 250
output_num_units = 4

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])
keep_prob = tf.placeholder(tf.float32)

# set remaining variables
epochs = 4000
batch_size = 1000
learning_rate = 0.001
### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'hidden_1': tf.Variable(tf.random_normal([input_num_units, hidden_num_units_1], seed=seed)),
    'hidden_2': tf.Variable(tf.random_normal([hidden_num_units_1, hidden_num_units_2], seed=seed)),
    'hidden_3': tf.Variable(tf.random_normal([hidden_num_units_2, hidden_num_units_3], seed=seed)),
    'hidden_4': tf.Variable(tf.random_normal([hidden_num_units_3, hidden_num_units_4], seed=seed)),
    'output': tf.Variable(tf.random_normal([hidden_num_units_4, output_num_units], seed=seed))
}

biases = {
    'hidden_1': tf.Variable(tf.random_normal([hidden_num_units_1], seed=seed)),
    'hidden_2': tf.Variable(tf.random_normal([hidden_num_units_2], seed=seed)),
    'hidden_3': tf.Variable(tf.random_normal([hidden_num_units_3], seed=seed)),
    'hidden_4': tf.Variable(tf.random_normal([hidden_num_units_4], seed=seed)),
    'output': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}


### Computational Graph

In [183]:
hidden_layer_1 = tf.add(tf.matmul(x, weights['hidden_1']), biases['hidden_1'])
hidden_layer_1 = tf.nn.relu(hidden_layer_1)
drop_out_layer_1 = tf.nn.dropout(hidden_layer_1, keep_prob)

hidden_layer_2 = tf.add(tf.matmul(drop_out_layer_1, weights['hidden_2']), biases['hidden_2'])
hidden_layer_2 = tf.nn.relu(hidden_layer_2)
# drop_out_layer_2 = tf.nn.dropout(hidden_layer_2, keep_prob)

# hidden_layer_3 = tf.add(tf.matmul(drop_out_layer_2, weights['hidden_3']), biases['hidden_3'])
# hidden_layer_3 = tf.nn.relu(hidden_layer_3)
# drop_out_layer_3 = tf.nn.dropout(hidden_layer_3, keep_prob)

# hidden_layer_4 = tf.add(tf.matmul(drop_out_layer_3, weights['hidden_4']), biases['hidden_4'])
# hidden_layer_4 = tf.nn.relu(hidden_layer_4)

output_layer = tf.matmul(hidden_layer_2, weights['output']) + biases['output']

### Cost function and Optimizer

In [184]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_layer))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [185]:
init = tf.global_variables_initializer()

In [186]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = int(train.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
            _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y, keep_prob : 0.5})
            
            avg_cost += c / total_batch
            
        print( "Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost) )
    
    print( "\nTraining complete!" )
    
    
    # find predictions on val set
    pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    print( "Validation Accuracy:", accuracy.eval({x: val_x.reshape(-1, input_num_units), 
                                                  y: dense_to_one_hot(val_y), 
                                                  keep_prob : 1.0}) )
    
    predict = tf.argmax(output_layer, 1)
    pred = predict.eval({x: test_x.reshape(-1, input_num_units), keep_prob : 1.0})

Epoch: 1 cost = 440.97443
Epoch: 2 cost = 421.91757
Epoch: 3 cost = 409.75943
Epoch: 4 cost = 372.05670
Epoch: 5 cost = 355.66632
Epoch: 6 cost = 346.14938
Epoch: 7 cost = 319.33624
Epoch: 8 cost = 286.10675
Epoch: 9 cost = 267.00604
Epoch: 10 cost = 259.26617
Epoch: 11 cost = 244.22218
Epoch: 12 cost = 212.65762
Epoch: 13 cost = 210.35611
Epoch: 14 cost = 186.58969
Epoch: 15 cost = 178.79897
Epoch: 16 cost = 178.20557
Epoch: 17 cost = 154.69800
Epoch: 18 cost = 149.12344
Epoch: 19 cost = 139.58783
Epoch: 20 cost = 135.91701
Epoch: 21 cost = 127.50242
Epoch: 22 cost = 129.14221
Epoch: 23 cost = 134.95364
Epoch: 24 cost = 120.09949
Epoch: 25 cost = 124.62045
Epoch: 26 cost = 115.15228
Epoch: 27 cost = 127.53633
Epoch: 28 cost = 128.52629
Epoch: 29 cost = 130.26202
Epoch: 30 cost = 132.38606
Epoch: 31 cost = 121.32672
Epoch: 32 cost = 118.45136
Epoch: 33 cost = 117.07609
Epoch: 34 cost = 106.00567
Epoch: 35 cost = 110.45853
Epoch: 36 cost = 113.68131
Epoch: 37 cost = 116.89716
Epoch: 38 

Epoch: 310 cost = 28.34150
Epoch: 311 cost = 25.42739
Epoch: 312 cost = 27.59896
Epoch: 313 cost = 25.61032
Epoch: 314 cost = 27.03467
Epoch: 315 cost = 25.02468
Epoch: 316 cost = 29.50532
Epoch: 317 cost = 25.10170
Epoch: 318 cost = 23.80391
Epoch: 319 cost = 23.27857
Epoch: 320 cost = 21.94126
Epoch: 321 cost = 29.10899
Epoch: 322 cost = 26.12933
Epoch: 323 cost = 25.79689
Epoch: 324 cost = 26.91681
Epoch: 325 cost = 28.68358
Epoch: 326 cost = 22.71417
Epoch: 327 cost = 26.13248
Epoch: 328 cost = 24.65426
Epoch: 329 cost = 24.93993
Epoch: 330 cost = 22.89246
Epoch: 331 cost = 27.49945
Epoch: 332 cost = 25.21330
Epoch: 333 cost = 21.98826
Epoch: 334 cost = 22.50724
Epoch: 335 cost = 25.90704
Epoch: 336 cost = 23.91768
Epoch: 337 cost = 24.76804
Epoch: 338 cost = 23.14163
Epoch: 339 cost = 23.87677
Epoch: 340 cost = 24.38708
Epoch: 341 cost = 26.64095
Epoch: 342 cost = 18.87297
Epoch: 343 cost = 22.34773
Epoch: 344 cost = 26.57303
Epoch: 345 cost = 21.56772
Epoch: 346 cost = 24.40390
E

Epoch: 617 cost = 13.33882
Epoch: 618 cost = 12.85783
Epoch: 619 cost = 16.66388
Epoch: 620 cost = 14.73934
Epoch: 621 cost = 11.77427
Epoch: 622 cost = 14.06375
Epoch: 623 cost = 15.02654
Epoch: 624 cost = 14.09764
Epoch: 625 cost = 12.31209
Epoch: 626 cost = 14.61698
Epoch: 627 cost = 13.28356
Epoch: 628 cost = 13.31483
Epoch: 629 cost = 14.00225
Epoch: 630 cost = 14.16218
Epoch: 631 cost = 12.80052
Epoch: 632 cost = 14.21000
Epoch: 633 cost = 14.41955
Epoch: 634 cost = 13.80234
Epoch: 635 cost = 13.75469
Epoch: 636 cost = 13.16094
Epoch: 637 cost = 12.82521
Epoch: 638 cost = 14.01319
Epoch: 639 cost = 12.89853
Epoch: 640 cost = 13.92361
Epoch: 641 cost = 13.10950
Epoch: 642 cost = 11.94674
Epoch: 643 cost = 14.14615
Epoch: 644 cost = 14.44314
Epoch: 645 cost = 13.05407
Epoch: 646 cost = 12.12584
Epoch: 647 cost = 11.13017
Epoch: 648 cost = 14.66244
Epoch: 649 cost = 14.90818
Epoch: 650 cost = 15.65917
Epoch: 651 cost = 12.60247
Epoch: 652 cost = 14.48329
Epoch: 653 cost = 13.89382
E

Epoch: 927 cost = 9.83172
Epoch: 928 cost = 9.14844
Epoch: 929 cost = 9.31527
Epoch: 930 cost = 9.96145
Epoch: 931 cost = 8.75988
Epoch: 932 cost = 8.78174
Epoch: 933 cost = 8.62730
Epoch: 934 cost = 8.67353
Epoch: 935 cost = 9.72016
Epoch: 936 cost = 9.91362
Epoch: 937 cost = 8.94307
Epoch: 938 cost = 8.96652
Epoch: 939 cost = 9.99731
Epoch: 940 cost = 8.31385
Epoch: 941 cost = 8.78750
Epoch: 942 cost = 6.95295
Epoch: 943 cost = 10.54563
Epoch: 944 cost = 8.63335
Epoch: 945 cost = 8.45638
Epoch: 946 cost = 11.84818
Epoch: 947 cost = 8.81106
Epoch: 948 cost = 8.12883
Epoch: 949 cost = 8.71102
Epoch: 950 cost = 9.84569
Epoch: 951 cost = 9.53437
Epoch: 952 cost = 9.16322
Epoch: 953 cost = 10.77121
Epoch: 954 cost = 8.17622
Epoch: 955 cost = 7.52287
Epoch: 956 cost = 9.79206
Epoch: 957 cost = 8.68572
Epoch: 958 cost = 9.54045
Epoch: 959 cost = 8.52187
Epoch: 960 cost = 8.25508
Epoch: 961 cost = 7.83659
Epoch: 962 cost = 8.08647
Epoch: 963 cost = 6.92499
Epoch: 964 cost = 7.76151
Epoch: 96

Epoch: 1235 cost = 6.09834
Epoch: 1236 cost = 7.22546
Epoch: 1237 cost = 5.55539
Epoch: 1238 cost = 6.31583
Epoch: 1239 cost = 5.51736
Epoch: 1240 cost = 6.59463
Epoch: 1241 cost = 5.39169
Epoch: 1242 cost = 7.55231
Epoch: 1243 cost = 7.87414
Epoch: 1244 cost = 6.79682
Epoch: 1245 cost = 6.87147
Epoch: 1246 cost = 7.55810
Epoch: 1247 cost = 7.10341
Epoch: 1248 cost = 7.66715
Epoch: 1249 cost = 7.46163
Epoch: 1250 cost = 6.86569
Epoch: 1251 cost = 7.95593
Epoch: 1252 cost = 5.62938
Epoch: 1253 cost = 5.70438
Epoch: 1254 cost = 5.64700
Epoch: 1255 cost = 7.92786
Epoch: 1256 cost = 6.34517
Epoch: 1257 cost = 7.39613
Epoch: 1258 cost = 5.53095
Epoch: 1259 cost = 8.22874
Epoch: 1260 cost = 5.63062
Epoch: 1261 cost = 6.22163
Epoch: 1262 cost = 7.23036
Epoch: 1263 cost = 6.88025
Epoch: 1264 cost = 5.93419
Epoch: 1265 cost = 6.72516
Epoch: 1266 cost = 5.26157
Epoch: 1267 cost = 7.12718
Epoch: 1268 cost = 7.40786
Epoch: 1269 cost = 4.51054
Epoch: 1270 cost = 6.03146
Epoch: 1271 cost = 5.65788
E

...

Epoch: 2761 cost = 2.48227
Epoch: 2762 cost = 1.90616
Epoch: 2763 cost = 1.60931
Epoch: 2764 cost = 1.78932
Epoch: 2765 cost = 2.07052
Epoch: 2766 cost = 1.51419
Epoch: 2767 cost = 1.76680
Epoch: 2768 cost = 1.70839
Epoch: 2769 cost = 1.59299
Epoch: 2770 cost = 2.02184
Epoch: 2771 cost = 1.92806
Epoch: 2772 cost = 2.07009
Epoch: 2773 cost = 1.63229
Epoch: 2774 cost = 2.04793
Epoch: 2775 cost = 2.06430
Epoch: 2776 cost = 2.19795
Epoch: 2777 cost = 1.97102
Epoch: 2778 cost = 2.09370
Epoch: 2779 cost = 2.52445
Epoch: 2780 cost = 2.00646
Epoch: 2781 cost = 1.95177
Epoch: 2782 cost = 1.96601
Epoch: 2783 cost = 2.01418
Epoch: 2784 cost = 2.45716
Epoch: 2785 cost = 2.08457
Epoch: 2786 cost = 2.10765
Epoch: 2787 cost = 2.33411
Epoch: 2788 cost = 1.47684
Epoch: 2789 cost = 1.86803
Epoch: 2790 cost = 1.81938
Epoch: 2791 cost = 1.88404
Epoch: 2792 cost = 1.71143
Epoch: 2793 cost = 1.70325
Epoch: 2794 cost = 1.41426
Epoch: 2795 cost = 1.84584
Epoch: 2796 cost = 1.71655
Epoch: 2797 cost = 2.14299
E

Epoch: 3065 cost = 1.74770
Epoch: 3066 cost = 1.77943
Epoch: 3067 cost = 1.35207
Epoch: 3068 cost = 1.85974
Epoch: 3069 cost = 1.93468
Epoch: 3070 cost = 1.52676
Epoch: 3071 cost = 1.55880
Epoch: 3072 cost = 1.66373
Epoch: 3073 cost = 1.88420
Epoch: 3074 cost = 1.56759
Epoch: 3075 cost = 1.48268
Epoch: 3076 cost = 1.66762
Epoch: 3077 cost = 1.65817
Epoch: 3078 cost = 1.76370
Epoch: 3079 cost = 1.91015
Epoch: 3080 cost = 1.54927
Epoch: 3081 cost = 1.69575
Epoch: 3082 cost = 1.39084
Epoch: 3083 cost = 1.68893
Epoch: 3084 cost = 1.78308
Epoch: 3085 cost = 2.13241
Epoch: 3086 cost = 1.50803
Epoch: 3087 cost = 1.92837
Epoch: 3088 cost = 2.07283
Epoch: 3089 cost = 1.63231
Epoch: 3090 cost = 1.55814
Epoch: 3091 cost = 1.47243
Epoch: 3092 cost = 1.55298
Epoch: 3093 cost = 1.27520
Epoch: 3094 cost = 1.43742
Epoch: 3095 cost = 1.08333
Epoch: 3096 cost = 1.70999
Epoch: 3097 cost = 1.21381
Epoch: 3098 cost = 1.61871
Epoch: 3099 cost = 1.00616
Epoch: 3100 cost = 1.93665
Epoch: 3101 cost = 1.61130
E

Epoch: 3369 cost = 1.26827
Epoch: 3370 cost = 1.26280
Epoch: 3371 cost = 1.52609
Epoch: 3372 cost = 1.74617
Epoch: 3373 cost = 1.29377
Epoch: 3374 cost = 1.30593
Epoch: 3375 cost = 1.36575
Epoch: 3376 cost = 1.12971
Epoch: 3377 cost = 1.42231
Epoch: 3378 cost = 1.00807
Epoch: 3379 cost = 1.20022
Epoch: 3380 cost = 1.19991
Epoch: 3381 cost = 1.17994
Epoch: 3382 cost = 1.54750
Epoch: 3383 cost = 1.35952
Epoch: 3384 cost = 1.18630
Epoch: 3385 cost = 0.93667
Epoch: 3386 cost = 1.01492
Epoch: 3387 cost = 1.07729
Epoch: 3388 cost = 1.75373
Epoch: 3389 cost = 1.20236
Epoch: 3390 cost = 1.31790
Epoch: 3391 cost = 0.94440
Epoch: 3392 cost = 1.29291
Epoch: 3393 cost = 1.17251
Epoch: 3394 cost = 1.69314
Epoch: 3395 cost = 1.56245
Epoch: 3396 cost = 1.50332
Epoch: 3397 cost = 1.40391
Epoch: 3398 cost = 1.31467
Epoch: 3399 cost = 1.37205
Epoch: 3400 cost = 1.27589
Epoch: 3401 cost = 1.12507
Epoch: 3402 cost = 1.27722
Epoch: 3403 cost = 1.09572
Epoch: 3404 cost = 1.32670
Epoch: 3405 cost = 1.19861
E

Epoch: 3675 cost = 0.85559
Epoch: 3676 cost = 1.22655
Epoch: 3677 cost = 1.27442
Epoch: 3678 cost = 1.05680
Epoch: 3679 cost = 1.20499
Epoch: 3680 cost = 1.08626
Epoch: 3681 cost = 0.88684
Epoch: 3682 cost = 1.03951
Epoch: 3683 cost = 1.14304
Epoch: 3684 cost = 1.31514
Epoch: 3685 cost = 1.01763
Epoch: 3686 cost = 1.09941
Epoch: 3687 cost = 1.40360
Epoch: 3688 cost = 1.17112
Epoch: 3689 cost = 1.34482
Epoch: 3690 cost = 1.47793
Epoch: 3691 cost = 1.02494
Epoch: 3692 cost = 1.03288
Epoch: 3693 cost = 1.00942
Epoch: 3694 cost = 0.59711
Epoch: 3695 cost = 1.40655
Epoch: 3696 cost = 1.44822
Epoch: 3697 cost = 0.91906
Epoch: 3698 cost = 1.22746
Epoch: 3699 cost = 0.99616
Epoch: 3700 cost = 0.96753
Epoch: 3701 cost = 0.95777
Epoch: 3702 cost = 0.97005
Epoch: 3703 cost = 0.91478
Epoch: 3704 cost = 1.02018
Epoch: 3705 cost = 0.86212
Epoch: 3706 cost = 0.95284
Epoch: 3707 cost = 0.83625
Epoch: 3708 cost = 0.81334
Epoch: 3709 cost = 1.43646
Epoch: 3710 cost = 0.98123
Epoch: 3711 cost = 1.30014
E

Epoch: 3982 cost = 0.98522
Epoch: 3983 cost = 0.60990
Epoch: 3984 cost = 0.82878
Epoch: 3985 cost = 0.82029
Epoch: 3986 cost = 1.07622
Epoch: 3987 cost = 0.89327
Epoch: 3988 cost = 0.81657
Epoch: 3989 cost = 0.99544
Epoch: 3990 cost = 0.99246
Epoch: 3991 cost = 0.88187
Epoch: 3992 cost = 0.87714
Epoch: 3993 cost = 0.84378
Epoch: 3994 cost = 1.07990
Epoch: 3995 cost = 0.85779
Epoch: 3996 cost = 0.79755
Epoch: 3997 cost = 1.12836
Epoch: 3998 cost = 1.23323
Epoch: 3999 cost = 0.78975
Epoch: 4000 cost = 1.12046

Training complete!
Validation Accuracy: 0.73006135


In [188]:
np.savetxt("prediction.csv", pred, fmt="%d",)